# Data Analysis Programming --- Assignment 2
## NYC Restaurants
                                         Tiffany Sung (from Group 3)

### Setup

In [ ]:
from pandas import Series, DataFrame
import pandas as pd
%pylab inline
df = pd.read_csv('NYC_Restaurants.csv', dtype=unicode)

### (Q1)

In [ ]:
df['RESTAURANT'] = df['DBA']+' '+df['BUILDING']+' '+df['STREET']+' '+df['ZIPCODE']

df['RESTAURANT'].iloc[0:10] #use iloc to locate first 10 rows

### (Q2)

In [ ]:
unique_restaurant = df.drop_duplicates(subset='RESTAURANT')
len(unique_restaurant)

### (Q3)
* Step 1: select the column 'DBA' and 'RESTAURANT'
* Step 2: drop the duplicates in 'RESTAURANT' then you'll have unique address data
* Step 3: make a mask that if values in 'DBA' still >1, meaning there's a chain
* Step 4: use the mask to find those 'DBA' which have chains
* Step 5: return length of chains which is the number of chains

In [ ]:
dba_restaurant = df[['DBA','RESTAURANT']]
dba_restaurant_dedup = dba_restaurant.drop_duplicates(subset='RESTAURANT')
chain_mask = dba_restaurant_dedup['DBA'].value_counts() > 1 
have_chains = dba_restaurant_dedup['DBA'].value_counts()[chain_mask]
len(have_chains)

### (Q4)

In [ ]:
have_chains[:20].plot(kind='bar')

### (Q5)
* Step 1: count the number of unique restaurants
* Step 2: count the number of unique restaurants that are chains
* Step 3: divided step2 by step3

In [ ]:
num_restaurant = len(unique_restaurant)
chains_sum = have_chains.sum()
print float(chains_sum)/(num_restaurant)

### (Q6)

* Step 1: set the 'Missing' in 'BORO' column to be NaN, that way it won't be counted
* Step 2: creat dataframe with 'DBA','BORO'and 'RESTAURANT'; drop the duplicates of 'RESTAURANT' return unique restaurants; and set 'DBA' as index
* Step 3: find the non-chains restaurants which chain_mask in (Q5), counts the numbers and plot

In [ ]:
#clean the data in column "BORO" where values are 'Missing'
boro_mask = (df['BORO'] == 'Missing')
df.loc[boro_mask, 'BORO'] = np.nan 

#create a new dataframe and put 'DBA', 'BORO' and 'RESTAURANT' into it.
dba_boro_restaurant = df[['DBA','BORO','RESTAURANT']].drop_duplicates(subset='RESTAURANT').set_index('DBA')

non_chains = dba_boro_restaurant['BORO'][~chain_mask].value_counts()
non_chains.plot(kind='bar')

### (Q7)
* Step 1: count the total restaurants in each borough
* Step 2: calculate the fraction by dividing non_chains by total restaurants in each borough

According to (Q6) and (Q7), the highest ratio of independent restaurant is in Brooklyn, but the borough that has the most independent retaurant is in Mahattan.

In [ ]:
total_restaurants_boro = dba_boro_restaurant['BORO'].value_counts() 
boro_fraction = non_chains / total_restaurants_boro
boro_fraction.plot(kind='bar')

### (Q8)

In [ ]:
cuisine = df[['CUISINE DESCRIPTION','RESTAURANT']].drop_duplicates(subset='RESTAURANT')
cuisine['CUISINE DESCRIPTION'].value_counts()[:20].plot(kind = 'bar')

### (Q9)
* Step 1: set a mask where the 'VIOLATION CODE' is null ; a mask where the 'VIOLATION CODE' is not null
* Step 2: apply the mask to violation_restaurant_df dataframe and drop duplicates of 'RESTAURANT'
* Step 3: set a mask that is the null_restaurants minus its intersection with not_restaurants
* Step 4: apply mask in step 3 to the null_restaurants and we get the truly no violation brands of restaurants

In [ ]:
violation_restaurant_df = df[['RESTAURANT', 'VIOLATION CODE', 'CUISINE DESCRIPTION']]

violations_null_mask = violation_restaurant_df['VIOLATION CODE'].isnull()
violations_not_mask = violation_restaurant_df['VIOLATION CODE'].notnull()

null_restaurants = violation_restaurant_df[violations_null_mask].drop_duplicates(subset='RESTAURANT')
not_restaurants = violation_restaurant_df[violations_not_mask].drop_duplicates(subset='RESTAURANT')

clean_mask = ~null_restaurants['RESTAURANT'].isin(not_restaurants['RESTAURANT'])

clean_restaurants = null_restaurants[clean_mask]
clean_restaurants['CUISINE DESCRIPTION'].value_counts()[:20].plot(kind='bar')

### (Q10)

In [ ]:
cuisine_20 = cuisine['CUISINE DESCRIPTION'].value_counts() >= 20
clean_restaurants.index = clean_restaurants['CUISINE DESCRIPTION']

no_violation_cuisine = clean_restaurants[cuisine_20]['CUISINE DESCRIPTION'].value_counts()
total_cuisine_restaurant = cuisine['CUISINE DESCRIPTION'].value_counts()[cuisine_20]

ratio = no_violation_cuisine*1.0 / total_cuisine_restaurant

print ratio.sort_values(ascending=False)[:10]
ratio.sort_values(ascending=False)[:10].plot(kind = 'bar')

### (Q11)
* Step 1: create a dataframe of boroughs and violations, dropping any missing values for boroughs
* Step 2: create a crosstab of the two columns
* Step 3: transpose such that 'BORO' is the row, and return the most common violation

In [ ]:
boro_violation = df[['BORO', 'VIOLATION DESCRIPTION']].dropna(axis = 0)
boro_violation_num = pd.crosstab(boro_violation['BORO'], boro_violation['VIOLATION DESCRIPTION'])
boro_violation_num.T.idxmax()

### (Q12)

Like the previous question, but divide the boro_violation_num with the total frenquency of each violations of each borough.

In [ ]:
violationFrequency = boro_violation['VIOLATION DESCRIPTION'].value_counts()
normalize = boro_violation_num / violationFrequency
normalize.T.idxmax()

### (Q13)

In [ ]:
zip_phone = df[['PHONE','ZIPCODE']]

def substract_code(s):
    return str(s)[:3]



zip_phone['AREA CODE'] = zip_phone['PHONE'].map(substract_code)

zip_phone = zip_phone[['ZIPCODE','AREA CODE']].drop_duplicates()
zip_phone_num = pd.crosstab(zip_phone['ZIPCODE'], zip_phone['AREA CODE'],margins = True).T

area_code_mask = zip_phone_num['All'] == 1
#zip_phone_num[area_code_mask].index
len(zip_phone_num[area_code_mask].index)

### (Q14)